<a href="https://colab.research.google.com/github/RyanUdeM/De-Bruijn-Graph/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle
import numpy as np


## Q1.1

In [52]:
# Q1.1
seed = 0
n_hidden = 2
dims = [10,2]
hidden_dims=(512, 256)

if seed is not None:
    np.random.seed(seed)

weights = {}
# self.weights is a dictionary with keys W1, b1, W2, b2, ..., Wm, Bm where m - 1 is the number of hidden layers
all_dims = [dims[0]] + list(hidden_dims) + [dims[1]]  ### all_dims = [input_dim, hidden1_dim, hidden2_dim, output_dim]
for layer_n in range(1, n_hidden + 2):
    # WRITE CODE HERE #############
    random_matrix = np.random.rand(all_dims[layer_n -1], all_dims[layer_n]) * 2 -1 ### random matrix [-1,1]
    W = random_matrix /np.sqrt(layer_n) 
    weights[f"W{layer_n}"] = W

    # ######################
    weights[f"b{layer_n}"] = np.zeros((1, all_dims[layer_n]))  ### b is a row vector has dimension of (1, hidden_dim)

print(all_dims)
print("b2 dim = ", weights['b2'].shape)
weights['b2'][0].shape
weights['W2'].shape

[10, 512, 256, 2]
b2 dim =  (1, 256)


(512, 256)

In [53]:
np.random.rand(2,3)*2 -1

array([[ 0.7863354 , -0.81775322, -0.25532018],
       [ 0.3572054 , -0.30865997, -0.11289668]])

# Q1.2

In [54]:
# Q1.2.a

def relu( x, grad=False):
    if grad:
        # WRITE CODE HERE
         return np.greater(x, 0).astype(int)
    # WRITE CODE HERE
    return np.maximum(0,x)

x = np.random.rand(2,3)*2 -1
print("x : ",x)
print("relu(x) : ", relu(x))
print("relu'(x) :", relu(x,grad = True))

x :  [[ 0.00514867 -0.09484145  0.8991575 ]
 [-0.43910339 -0.98173638  0.04652703]]
relu(x) :  [[0.00514867 0.         0.8991575 ]
 [0.         0.         0.04652703]]
relu'(x) : [[1 0 1]
 [0 0 1]]


In [55]:
def sigmoid(x, grad=False):
    sig = 1/(1+np.exp(-x))
    if grad:
        # WRITE CODE HERE
        return sig*(1-sig)
        # WRITE CODE HERE
    return sig


x = np.random.rand(2,3)*2 -1
print("x : ",x)
print("sigmoid(x) : ", sigmoid(x))
print("sigmoid'(x) :", sigmoid(x,grad = True))

x :  [[-0.2146626   0.77727255 -0.08572641]
 [-0.0749824   0.27609295  0.90786553]]
sigmoid(x) :  [[0.44653948 0.68509199 0.47858151]
 [0.48126318 0.5685881  0.71256318]]
sigmoid'(x) : [[0.24714197 0.21574096 0.24954125]
 [0.24964893 0.24529567 0.20481689]]


In [56]:
def tanh( x, grad=False):
    tanh = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    if grad:
        # WRITE CODE HERE
        return 1-tanh**2
    # WRITE CODE HERE
    return tanh

x = np.random.rand(2,3)*2 -1
print("x : ",x)
print("tanh(x) : ", tanh(x))
print("tanh'(x) :", tanh(x,grad = True))


x :  [[-0.51122554  0.89105113 -0.95641943]
 [ 0.63345782 -0.18957018 -0.31317862]]
tanh(x) :  [[-0.47089953  0.71191252 -0.74267549]
 [ 0.5604286  -0.1873315  -0.30332604]]
tanh'(x) : [[0.77825363 0.49318057 0.44843312]
 [0.68591979 0.96490691 0.90799332]]


In [0]:
def activation(self, x, grad=False):
    if self.activation_str == "relu":
        return relu(x, grad)
    
    elif self.activation_str == "sigmoid":
        return sigmoid(x, grad)

    elif self.activation_str == "tanh":
        return tanh(x, grad)

    else:
        raise Exception("invalid")
    return 0

## Q1.3

In [0]:
def softmax( x):
    # Remember that softmax(x-C) = softmax(x) when C is a constant.
    # WRITE CODE HERE
    e_x = np.exp(x.T - np.max(x, axis = -1))
    return (e_x / e_x.sum(axis=0)).T

In [114]:
print("############### 2D ##############")
scores2D = np.array([[1, 2, 3, 6],
                     [2, 4, 5, 6],
                     [3, 8, 7, 6]])

sfm = softmax(scores2D)
print(sfm)
print(np.sum(sfm[1]))

print("input dim :", scores2D.shape)
print("softmax dim: ", sfm.shape)

print("############### 1D ##############")
scores1D = np.array([1, 2, 3, 6])
sfm1 = softmax(scores1D)
print(sfm1)
print(np.sum(sfm1))

print("input dim :", scores1D.shape)
print("softmax dim: ", sfm1.shape)

############### 2D ##############
[[0.00626879 0.01704033 0.04632042 0.93037047]
 [0.01203764 0.08894682 0.24178252 0.65723302]
 [0.00446236 0.66227241 0.24363641 0.08962882]]
1.0
input dim : (3, 4)
softmax dim:  (3, 4)
############### 1D ##############
[0.00626879 0.01704033 0.04632042 0.93037047]
1.0
input dim : (4,)
softmax dim:  (4,)


In [0]:
class NN(object):
    def __init__(self,
                 hidden_dims=(512, 256),
                 datapath='cifar10.pkl',
                 n_classes=10,
                 epsilon=1e-6,
                 lr=7e-4,
                 batch_size=1000,
                 seed=None,
                 activation="relu",
                 init_method="glorot"
                 ):

        self.hidden_dims = hidden_dims
        self.n_hidden = len(hidden_dims)
        self.datapath = datapath
        self.n_classes = n_classes
        self.lr = lr
        self.batch_size = batch_size
        self.init_method = init_method
        self.seed = seed
        self.activation_str = activation
        self.epsilon = epsilon

        self.train_logs = {'train_accuracy': [], 'validation_accuracy': [], 'train_loss': [], 'validation_loss': []}

        if datapath is not None:
            u = pickle._Unpickler(open(datapath, 'rb'))
            u.encoding = 'latin1'
            self.train, self.valid, self.test = u.load()
        else:
            self.train, self.valid, self.test = None, None, None

    # Q1.1
    def initialize_weights(self, dims):
        if self.seed is not None:
            np.random.seed(self.seed)

        self.weights = {}
        # self.weights is a dictionary with keys W1, b1, W2, b2, ..., Wm, Bm where m - 1 is the number of hidden layers
        all_dims = [dims[0]] + list(self.hidden_dims) + [dims[1]] ### all_dims = [input_dim, hidden1_dim, hidden2_dim, output_dim]
        for layer_n in range(1, self.n_hidden + 2):
            # WRITE CODE HERE #############
            random_matrix = np.random.rand(all_dims[layer_n -1], all_dims[layer_n]) * 2 -1 ### random matrix dim: (dim_(h-1), dim_h), value range: [-1,1]
            W = random_matrix /np.sqrt(layer_n) 
            weights[f"W{layer_n}"] = W

            # ######################
            self.weights[f"b{layer_n}"] = np.zeros((1, all_dims[layer_n])) #### row vector dim: (1, dh)

    # Q1.2.a
    def relu(self, x, grad=False):
        if grad:
            # WRITE CODE HERE
            return np.greater(x, 0).astype(int)
            # WRITE CODE HERE
        return np.maximum(0,x)

    # Q1.2.b
    def sigmoid(self, x, grad=False):
        sig = 1/(1+np.exp(-x))
        if grad:
            # WRITE CODE HERE
            return sig*(1-sig)
            # WRITE CODE HERE
        return sig

    # Q1.2.c
    def tanh(self, x, grad=False):
        tanh = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
        if grad:
            # WRITE CODE HERE
            return 1-tanh**2
        # WRITE CODE HERE
        return tanh

    # Q1.2.d
    def activation(self, x, grad=False):
        if self.activation_str == "relu":
            return self.relu(x, grad)

        elif self.activation_str == "sigmoid":
            return self.sigmoid(x, grad)

        elif self.activation_str == "tanh":
            return self.tanh(x, grad)
        else:
            raise Exception("invalid")
        return 0

    # Q1.3
    def softmax(self, x):
        # Remember that softmax(x-C) = softmax(x) when C is a constant.
        # WRITE CODE HERE
        e_x = np.exp(x.T - np.max(x, axis = -1))
        return (e_x / e_x.sum(axis=0)).T

    def forward(self, x):
        cache = {"Z0": x}
        # cache is a dictionary with keys Z0, A0, ..., Zm, Am where m - 1 is the number of hidden layers
        # Ai corresponds to the preactivation at layer i, Zi corresponds to the activation at layer i
        # WRITE CODE HERE
        pass
        return cache

    def backward(self, cache, labels):
        output = cache[f"Z{self.n_hidden + 1}"]
        grads = {}
        # grads is a dictionary with keys dAm, dWm, dbm, dZ(m-1), dA(m-1), ..., dW1, db1
        # WRITE CODE HERE
        pass
        return grads

    def update(self, grads):
        for layer in range(1, self.n_hidden + 2):
            # WRITE CODE HERE
            pass

    def one_hot(self, y):
        # WRITE CODE HERE
        pass
        return 0

    def loss(self, prediction, labels):
        prediction[np.where(prediction < self.epsilon)] = self.epsilon
        prediction[np.where(prediction > 1 - self.epsilon)] = 1 - self.epsilon
        # WRITE CODE HERE
        pass
        return 0

    def compute_loss_and_accuracy(self, X, y):
        one_y = self.one_hot(y)
        cache = self.forward(X)
        predictions = np.argmax(cache[f"Z{self.n_hidden + 1}"], axis=1)
        accuracy = np.mean(y == predictions)
        loss = self.loss(cache[f"Z{self.n_hidden + 1}"], one_y)
        return loss, accuracy, predictions

    def train_loop(self, n_epochs):
        X_train, y_train = self.train
        y_onehot = self.one_hot(y_train)
        dims = [X_train.shape[1], y_onehot.shape[1]]
        self.initialize_weights(dims)

        n_batches = int(np.ceil(X_train.shape[0] / self.batch_size))

        for epoch in range(n_epochs):
            for batch in range(n_batches):
                minibatchX = X_train[self.batch_size * batch:self.batch_size * (batch + 1), :]
                minibatchY = y_onehot[self.batch_size * batch:self.batch_size * (batch + 1), :]
                # WRITE CODE HERE
                pass

            X_train, y_train = self.train
            train_loss, train_accuracy, _ = self.compute_loss_and_accuracy(X_train, y_train)
            X_valid, y_valid = self.valid
            valid_loss, valid_accuracy, _ = self.compute_loss_and_accuracy(X_valid, y_valid)

            self.train_logs['train_accuracy'].append(train_accuracy)
            self.train_logs['validation_accuracy'].append(valid_accuracy)
            self.train_logs['train_loss'].append(train_loss)
            self.train_logs['validation_loss'].append(valid_loss)

        return self.train_logs

    def evaluate(self):
        X_test, y_test = self.test
        # WRITE CODE HERE
        pass
        return 0